In [1]:
%matplotlib qt
import numpy as np
import h5py
import pickle
import os.path as op
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [2]:
f = pickle.load( open( "epla_P04S01_.p", "rb" ) )
# f = pickle.load( open( "epla_P02S04_.p", "rb" ) )
epochs = f["epochs"]
labels = f["labels"]
real_word = ['C','A','L','I','B','R','A','T','I','O','N']

In [10]:
count_1 = 0
count_2 = 0
count_3 = 0
mask = np.zeros(epochs[0].shape[2])
l = np.linspace(0,1,20)
mask[20:40] = 1
mask[:20] = l
l = np.linspace(1,0,len(mask)-40)
mask[40:] = l
#mask = np.ones(ed.shape[2])

for i in range(len(epochs)):
    e = epochs[i]
    lab = labels[i]
    fig = plt.figure()
    lab = labels[i]
    M = np.zeros([e.shape[1]*e.shape[2],e.shape[0]])
    for j in range(e.shape[0]):
        tmp = e[j].copy()
        for t in range(tmp.shape[0]):
            tmp[t,:] = tmp[t,:] * mask
        M[:,j] = np.reshape(tmp,e.shape[1]*e.shape[2])
    u,s,v = np.linalg.svd(M/np.linalg.norm(M),full_matrices=False)
    tmp = np.cumsum(s**2) - 1
    n = np.zeros(len(tmp))
    for j in range(len(tmp)):
        n[j] = np.linalg.norm(np.array([float(j)/len(tmp),tmp[j]]),1)
    idx = np.argmin(n)
    M = u[:,:idx].T.dot(M)
    n = np.zeros(len(lab))
    for li, key in enumerate(lab):
        l = lab[key]
        idx_t = np.where(l==0)[0]
        idx_nt = np.where(l==1)[0]
        M_ref = np.mean(M[:,idx_t],axis=1)
        X = M.T.copy()
        y = np.zeros(X.shape[0])
        y[idx_t] = 1
        M_nt_mean = np.mean(X[idx_nt],axis=0)
        
        clf = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto').fit(X, y)       
        
        n[li] = clf.predict_proba(np.atleast_2d(M_ref))[0,1]
        if key==real_word[i]:
            z = li
            C = clf.covariance_.copy()
            u,s,v = np.linalg.svd(C)
            s = 1./np.sqrt(s)
            P = u.dot(np.diag(s))
            X_t = X[idx_t,:].dot(P)
            X_nt = X[idx_nt,:].dot(P)
            v_proj = np.mean(X_t,axis=0) - np.mean(X_nt,axis=0)
            X_t = X_t.dot(v_proj.T)
            X_nt = X_nt.dot(v_proj.T)
            plt.plot(X_nt,np.ones(len(X_nt)),'b.')
            plt.plot(X_t,np.ones(len(X_t)),'r.')
    idx = np.argsort(n)[::-1]
    n = n[idx]
    z2 = np.where(idx==z)[0]
    if z2==0:
        count_1 += 1
    if z2<=1:
        count_2 += 1
    if z2<=2:
        count_3 += 1
#     plt.plot(n,'b.-')
#     plt.plot(z2,n[z2],'r*')
    plt.show()
print count_1, count_2, count_3            

10 10 10
